# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [1]:
from copy import deepcopy
from bac.simulate.coding import Encoder
import bac.simulate.gromacs as gmx

In [9]:
em_steep = gmx.Run(integrator='steep', number_of_steps=5000)
em_steep.output_name = 'em_steep'

In [3]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10

In [4]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.non_bonded_controller.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 1
nbc.neighbor_list_type = gmx.non_bonded_controller.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.non_bonded_controller.PeriodicBoundaryConditionType.xyz
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.non_bonded_controller.CoulombType.pme
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_type = gmx.non_bonded_controller.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.non_bonded_controller.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.non_bonded_controller.LongRangeDispersionCorrectionType.energy_and_pressure

In [5]:
fe = gmx.FreeEnergyController()

fe.initial_lambda_state = 0
fe.delta_lambda = 0
fe.calculate_lambda_neighbors = 1
fe.van_der_waals_lambdas = [f'{x*0.05:.2f}' for x in range(21)] 
fe.coulomb_lambdas = [f'{0:.2f}']*21
fe.bonded_lambdas = [f'{0:.2f}']*21
fe.restraint_lambdas = [f'{0:.2f}']*21
fe.mass_lambdas = [f'{0:.2f}']*21
fe.temperature_lambdas = [f'{0:.2f}']*21

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

em_steep.free_energy_controller = fe

In [6]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

In [24]:
Encoder.encode(em_steep, engine=Engine.gromacs, path='/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/em_steep.mdp')

In [10]:
em_bfgs = next(em_steep)

In [8]:
em_bfgs.integrator = gmx.Integrator.l_bfgs
em_bfgs.constraints.type = gmx.ConstraintType.none

In [9]:
nvt = next(em_bfgs)

In [10]:
nvt.integrator = 'sd'
nvt.delta_time = 0.002
nvt.number_of_steps = 50000
nvt.com_motion_removal_frequency = 100

nvt.generate_velocities = True
nvt.generate_temperature = 300

In [11]:
nvt.temperature_controller.groups = 'system'
nvt.temperature_controller.time = 1
nvt.temperature_controller.temperature = 300

In [13]:
Encoder.encode(nvt, path='/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/nvt.mdp')

In [30]:
for lambda_state in range(21):
    em_steep.free_energy_controller.initial_lambda_state = lambda_state
    Encoder.encode(em_steep, path=f'/Users/kristofarkas/Developer/BAC2/docs/gromacs_fe_example/em_steep/em_steep_{lambda_state:02d}.mdp')

## Operation and OperationQueue

In [2]:
from bac.simulate import Operation

In [3]:
em_steep = gmx.Run()

In [4]:
em_steep.output_name = 'min_0'
em_steep.coordinates = 'Methane/methane_water.gro'
em_steep.topology = 'Methane/topol.top'

In [5]:
Operation(em_steep).execute_path

[['gmx',
  'grompp',
  '-f',
  '51ef0605-48e9-44e2-9082-13c55c53881c.mpd',
  '-c',
  'Methane/methane_water.gro',
  '-p',
  'Methane/topol.top',
  '-o',
  'min_0.tpr'],
 ['gmx', 'mdrun', '-nt', 1, '-deffnm', 'min_0']]

In [6]:
em_lbfgs = next(em_steep)
em_lbfgs.output_name = 'min0'

In [7]:
Operation(em_lbfgs).execute_path

[['gmx',
  'grompp',
  '-f',
  'a62a64db-2331-45ff-815e-2649247eb603.mpd',
  '-c',
  'min_0.gro',
  '-p',
  'Methane/topol.top',
  '-o',
  'min0.tpr'],
 ['gmx', 'mdrun', '-nt', 1, '-deffnm', 'min0']]

In [8]:
nvt = next(em_lbfgs)
nvt.output_name = 'nvt0'
Operation(nvt).execute_path

[['gmx',
  'grompp',
  '-f',
  '4c6de065-17b4-4392-958f-449e7bb91092.mpd',
  '-c',
  'min0.gro',
  '-p',
  'Methane/topol.top',
  '-o',
  'nvt0.tpr'],
 ['gmx', 'mdrun', '-nt', 1, '-deffnm', 'nvt0']]

In [9]:
nvt.constraints.continuation = True

In [10]:
npt = next(nvt)

In [11]:
npt.output_name = 'npt0'

In [12]:
Operation(npt).execute_path

[['gmx',
  'grompp',
  '-f',
  'daf58c4e-0440-41e7-8e6b-c56f75c00f6f.mpd',
  '-c',
  'nvt0.gro',
  '-p',
  'Methane/topol.top',
  '-o',
  'npt0.tpr',
  '-t',
  'nvt0.cpt'],
 ['gmx', 'mdrun', '-nt', 1, '-deffnm', 'npt0']]